In [167]:
import pandas as pd
import numpy as np

In [226]:
base_rate = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\3_base_rate.csv")
territory_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\4_Territory_factor_Rating_Area_Factors_USAA_Group_Owners_Forms.csv")
fire_protection_class_construction_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\5_9_Protection_construction_factor_owner.csv")
form_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\6_form_factor.csv")
tier_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\7_tier_factor.csv")
amount_of_insurance_deductible = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\8_amount_of_insurance_factor.csv")
age_of_home = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\10_age_of_home.csv")
insured_age_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\15_Insured_age_factor.csv"))
roof_factor_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\22_roof_factor.csv"))
sqftxnum_stories_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\23_sq_ftxnum_stories.csv"))
sqftxnum_stories_table['Max sq ft'] = sqftxnum_stories_table['Max sq ft'].replace('>10000', 15000).astype(int)
garage_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\26_Garage.csv"))
home_protector_coverage_table = pd.read_csv(("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\27_home_protector_coverage.csv"))


In [227]:
home_protector_coverage_table

,Form,Other Perils,Fire,Other Wind and Hail,Theft,Water,Liability,Hurricane,Severe Thunderstorm,Winter Storm,Earthquake
0,HO-3R,1.0074,1.0074,1.0373,1,1,1,1.0569,1.0025,1.0066,1.0074
1,HO-9R,1.0000,1.0000,1.0000,1,1,1,1.0000,1.0000,1.0000,1.0000


In [219]:
insured_age_table.dtypes

Insured Age             object
Other Perils           float64
Fire                   float64
Other Wind and Hail    float64
Theft                  float64
Water                  float64
Liability              float64
Hurricane              float64
Severe Thunderstorm    float64
Winter Storm           float64
Earthquake             float64
dtype: object

In [187]:
base_rate = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\3_base_rate.csv")
territory_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\4_Territory_factor_Rating_Area_Factors_USAA_Group_Owners_Forms.csv")
fire_protection_class_construction_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\5_9_Protection_construction_factor_owner.csv")
form_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\6_form_factor.csv")
tier_factor = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\7_tier_factor.csv")
amount_of_insurance_deductible = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\8_amount_of_insurance_factor.csv")



def amount_of_insurance_deductible_factor(amount_of_insurance_df, amount_of_insurance, deductible, aep):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = amount_of_insurance_df[(amount_of_insurance_df["Amount of Insurance"] == amount_of_insurance) & (amount_of_insurance_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[deductible].values[0] if deductible in peril_row.columns else np.nan
            
            # If the deductible value is NaN, try the aep column
            if pd.isna(value) and aep in peril_row.columns:
                value = peril_row[aep].values[0]
            
            # If both deductible and aep are NaN, take the value from the "10%" column
            if pd.isna(value) and "10%" in peril_row.columns:
                value = peril_row["10%"].values[0]
            
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value
    
    return result



def calculate_insurance_premium(rating_area_value, construction_type, protection_class , form, tier, amount_of_insurance_df, amount_of_insurance,
                                deductible, aep):
    # Filter the row for the specified rating area in df_rating_area
    rating_area_row = territory_factor[territory_factor['Rating Area'] == rating_area_value]
    
    # Filter the row for the specified construction type and protection class in df_fire_protection
    fire_protection_class_construction_factor_row = fire_protection_class_construction_factor[(fire_protection_class_construction_factor['Construction Type'] == construction_type) & 
                                        (fire_protection_class_construction_factor['Protection Class'] == protection_class)]
    
    form_factor_row = form_factor[form_factor['Form'] == form]

    tier_factor_row = tier_factor[tier_factor['Tier'] == tier]


    rating_area_values = rating_area_row.drop(columns="Rating Area").iloc[0]
    fire_protection_class_construction_factor_values = fire_protection_class_construction_factor_row.drop(columns=["Construction Type", "Protection Class"]).iloc[0]
    form_factor_values = form_factor_row.drop(columns="Form").iloc[0]
    tier_factor_values =  tier_factor_row.drop(columns=["Tier", "Other Optional Coverages"]).iloc[0]

    amount_of_insurance_deductible_factor_values = amount_of_insurance_deductible_factor(amount_of_insurance_df, amount_of_insurance, deductible, aep)

        # Calculating final values by multiplying rating area, protection factors, and transposed DataFrame values
    result = rating_area_values * fire_protection_class_construction_factor_values * form_factor_values * tier_factor_values * amount_of_insurance_deductible_factor_values  * base_rate.iloc[0]
        
    dataframe = result.T

    return dataframe



def user_inputs(rating_area_value=701, construction_type="Masonry Veneer", 
                protection_class=3, form="HO-3R", tier = 30, 
                amount_of_insurance_df = amount_of_insurance_deductible, amount_of_insurance = '$200,000', 
                deductible = '$500', aep = '2%' ):
        
        # Prompting for inputs with default values
        rating_area_value = int(input(f"Please enter the Rating Area [{rating_area_value}]: ") or rating_area_value)
        construction_type = input(f"Please enter the Construction Type [{construction_type}]: ") or construction_type
        protection_class = int(input(f"Please enter the Protection Class [{protection_class}]: ") or protection_class)
        form = input(f"Please enter the Form [{form}]: ") or form
        tier = int(input(f"Please enter the Form [{tier}]: ") or tier)
        amount_of_insurance_df = amount_of_insurance_df,
        amount_of_insurance = input(f"Please enter the amount of insurance [{amount_of_insurance}]: ") or amount_of_insurance
        deductible = input(f"Please enter the deductible [{deductible}]: ") or deductible
        aep = input(f"Please enter the annual exceedence probability [{aep}]: ") or aep

        # Running the calculation function with the provided inputs
        result = calculate_insurance_premium(rating_area_value, construction_type, 
                                             protection_class, form, tier, 
                                             amount_of_insurance_df, amount_of_insurance, deductible, aep)

        return result.values.sum()

In [ ]:
user_inputs()

In [ ]:
amount_of_insurance_deductible

In [139]:
# Function to prompt for rating area, construction type, and protection class, 
# then calculate based on the provided tables and the transposed DataFrame

def amount_of_insurance_deductible_factor(df, amount_of_insurance, deductible, aep):
  # Define the specific perils to filter
  perils = [
    "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
    "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
  ]
  
  # Initialize the result DataFrame with perils as columns and a single row for the values
  result = pd.DataFrame(columns=perils, index=[0])
  
  # Filter the dataframe based on the given amount_of_insurance
  filtered_df = df[(df["Amount of Insurance"] == amount_of_insurance) & (df["Peril"].isin(perils))]
  
  # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
  for peril in perils:
    # Filter the row for the current peril
    peril_row = filtered_df[filtered_df["Peril"] == peril]
  
  # Check if the peril_row is non-empty
  if not peril_row.empty:
    # First try to retrieve the value from the deductible column
    value = peril_row[deductible].values[0] if deductible in peril_row.columns else np.nan
  
  # If the deductible value is NaN, try the aep column
  if pd.isna(value) and aep in peril_row.columns:
    value = peril_row[aep].values[0]
  
  # If both deductible and aep are NaN, take the value from the "10%" column
  if pd.isna(value) and "10%" in peril_row.columns:
    value = peril_row["10%"].values[0]
  
  # Assign the final value to the result DataFrame
  result[peril].iloc[0] = value
  
  return result        
  
  
def calculate_insurance_premium(rating_area_value, construction_type, protection_class , form, tier, df, amount_of_insurance,
                                  deductible, aep):
    # Filter the row for the specified rating area in df_rating_area
    rating_area_row = territory_factor[territory_factor['Rating Area'] == rating_area_value]
  
  # Filter the row for the specified construction type and protection class in df_fire_protection
    fire_protection_class_construction_factor_row = fire_protection_class_construction_factor[(fire_protection_class_construction_factor['Construction Type'] == construction_type) & 
                                                                                              (fire_protection_class_construction_factor['Protection Class'] == protection_class)]
  
    form_factor_row = form_factor[form_factor['Form'] == form]
  
    tier_factor_row = tier_factor[tier_factor['Tier'] == tier]
  
  # Check if both rows exist
    if not rating_area_row.empty and not fire_protection_class_construction_factor_row.empty and not form_factor_row.empty and not tier_factor_row.empty :
    # Dropping non-numeric columns to get values for multiplication
        rating_area_values = rating_area_row.drop(columns="Rating Area").iloc[0]
        fire_protection_class_construction_factor_values = fire_protection_class_construction_factor_row.drop(columns=["Construction Type", "Protection Class"]).iloc[0]
        form_factor_values = form_factor_row.drop(columns="Form").iloc[0]
        tier_factor_values =  tier_factor_row.drop(columns=["Tier", "Other Optional Coverages"]).iloc[0]
  
        amount_of_insurance_deductible_factor_values = amount_of_insurance_deductible_factor(df, amount_of_insurance, deductible, aep)
  
  # Calculating final values by multiplying rating area, protection factors, and transposed DataFrame values
        result = rating_area_values * fire_protection_class_construction_factor_values * form_factor_values * tier_factor_values * amount_of_insurance_deductible_factor_values  * base_rate.iloc[0]
  
        dataframe = result.T
  
        return dataframe
  
  
  
def user_inputs(rating_area_value=701, construction_type="Masonry Veneer", 
                  protection_class=3, form="HO-3R", tier = 30, 
                  df = amount_of_insurance_deductible, amount_of_insurance = '$200,000', 
                  deductible = '$500', aep = '2%' ):
    
    # Prompting for inputs with default values
    rating_area_value = int(input(f"Please enter the Rating Area [{rating_area_value}]: ") or rating_area_value)
    construction_type = input(f"Please enter the Construction Type [{construction_type}]: ") or construction_type
    protection_class = int(input(f"Please enter the Protection Class [{protection_class}]: ") or protection_class)
    form = input(f"Please enter the Form [{form}]: ") or form
    tier = int(input(f"Please enter the Form [{tier}]: ") or tier)
    df = df,
    amount_of_insurance = input(f"Please enter the amount of insurance [{amount_of_insurance}]: ") or amount_of_insurance
    deductible = input(f"Please enter the deductible [{deductible}]: ") or deductible
    aep = input(f"Please enter the annual exceedence probability [{aep}]: ") or aep
  
  # Running the calculation function with the provided inputs
    result = calculate_insurance_premium(rating_area_value, construction_type, protection_class, form, tier, df, amount_of_insurance, deductible, aep)
  
    return result.values.sum()

In [ ]:
user_inputs()

In [192]:
def amount_of_insurance_deductible_factor(amount_of_insurance_df, amount_of_insurance, deductible, aep):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = amount_of_insurance_df[(amount_of_insurance_df["Amount of Insurance"] == amount_of_insurance) & (amount_of_insurance_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[deductible].values[0] if deductible in peril_row.columns else np.nan
            
            # If the deductible value is NaN, try the aep column
            if pd.isna(value) and aep in peril_row.columns:
                value = peril_row[aep].values[0]
            
            # If both deductible and aep are NaN, take the value from the "10%" column
            if pd.isna(value) and "10%" in peril_row.columns:
                value = peril_row["10%"].values[0]
            
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value
    
    return result

In [194]:
def amount_of_insurance_deductible_factor(amount_of_insurance_df, amount_of_insurance, deductible, aep):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = amount_of_insurance_df[(amount_of_insurance_df["Amount of Insurance"] == amount_of_insurance) & (amount_of_insurance_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[deductible].values[0] if deductible in peril_row.columns else np.nan
            
            # If the deductible value is NaN, try the aep column
            if pd.isna(value) and aep in peril_row.columns:
                value = peril_row[aep].values[0]
            
            # If both deductible and aep are NaN, take the value from the "10%" column
            if pd.isna(value) and "10%" in peril_row.columns:
                value = peril_row["10%"].values[0]
            
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value
    
    # Adjust the sixth row based on the amount_of_insurance conditions
    if amount_of_insurance in ["$10,000", "$50,000", "$100,000", "$150,000", "$200,000", "$250,000", "$300,000"]:
        result.iloc[0, 5] *= 0.8796
    elif amount_of_insurance in ["$350,000", "$400,000", "$450,000", "$500,000"]:
        result.iloc[0, 5] *= 1.1489
    elif amount_of_insurance in ["$600,000", "$750,000", "$1,000,000", "$1,500,000", "$2,000,000", "$3,000,000"]:
        result.iloc[0, 5] *= 1.5144

    return result

In [195]:
amount_of_insurance_deductible_factor(amount_of_insurance_deductible, '$200,000', '$500', '2%')

,Other Perils,Fire,Other Wind and Hail,Theft,Water,Liability,Hurricane,Severe Thunderstorm,Winter Storm,Earthquake
0,1.5489,0.9867,0.733,1.7425,1.595,0.8796,0.9635,0.6175,0.7226,1.0


In [208]:
def roof_factor(roof_factor_df, roof_type, impact_resistant_class, roof_age):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = roof_factor_df[(roof_factor_df["Roof Type"] == roof_type) & (roof_factor_df["Impact Resistant Class Code"] == impact_resistant_class) & (roof_factor_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[roof_age].values[0] if roof_age in peril_row.columns else np.nan
                       
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value

    return result

In [210]:
roof_factor(roof_factor_table, 'Fiberglass Shingle', 0, '9')

,Other Perils,Fire,Other Wind and Hail,Theft,Water,Liability,Hurricane,Severe Thunderstorm,Winter Storm,Earthquake
0,1.0274,1.0,2.1141,1.0,1.0,1.0,1.0,2.0355,1.0,1.0


In [220]:
def sqftxnum_stories(sqftxnum_stories_df, sqft, num_stories):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    if sqft > 10000:
        filtered_df = sqftxnum_stories_df[sqftxnum_stories_df['Min sq ft'] == 10000]
    else:
        filtered_df = sqftxnum_stories_df[(sqftxnum_stories_df['Min sq ft'] <= sqft) & (sqftxnum_stories_df['Max sq ft'] >= sqft)]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[num_stories].values[0] if num_stories in peril_row.columns else np.nan
                       
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value

    return result

In [221]:
sqftxnum_stories(sqftxnum_stories_table, 1800, '1')

,Other Perils,Fire,Other Wind and Hail,Theft,Water,Liability,Hurricane,Severe Thunderstorm,Winter Storm,Earthquake
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [224]:
def garage(garage_df, garage_size, garage_type):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = garage_df[(garage_df["Garage Size"] == garage_size) & (garage_df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[garage_type].values[0] if garage_type in peril_row.columns else np.nan
                       
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value

    return result

In [225]:
garage(garage_table, '2 Car', 'Attached')

,Other Perils,Fire,Other Wind and Hail,Theft,Water,Liability,Hurricane,Severe Thunderstorm,Winter Storm,Earthquake
0,1.0,1.0,1.1288,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## Preserve this

In [165]:
# Function to prompt for rating area, construction type, and protection class, 
# then calculate based on the provided tables and the transposed DataFrame


def calculate_insurance_premium(rating_area_value, construction_type, protection_class , form, tier):
    # Filter the row for the specified rating area in df_rating_area
    rating_area_row = territory_factor[territory_factor['Rating Area'] == rating_area_value]
    
    # Filter the row for the specified construction type and protection class in df_fire_protection
    fire_protection_class_construction_factor_row = fire_protection_class_construction_factor[(fire_protection_class_construction_factor['Construction Type'] == construction_type) & 
                                        (fire_protection_class_construction_factor['Protection Class'] == protection_class)]
    
    form_factor_row = form_factor[form_factor['Form'] == form]

    tier_factor_row = tier_factor[tier_factor['Tier'] == tier]


    rating_area_values = rating_area_row.drop(columns="Rating Area").iloc[0]
    fire_protection_class_construction_factor_values = fire_protection_class_construction_factor_row.drop(columns=["Construction Type", "Protection Class"]).iloc[0]
    form_factor_values = form_factor_row.drop(columns="Form").iloc[0]
    tier_factor_values =  tier_factor_row.drop(columns=["Tier", "Other Optional Coverages"]).iloc[0]

    

        # Calculating final values by multiplying rating area, protection factors, and transposed DataFrame values
    result = rating_area_values * fire_protection_class_construction_factor_values * form_factor_values * tier_factor_values *  base_rate.iloc[0]
        
    dataframe = result.T

    return dataframe



def user_inputs(rating_area_value=701, construction_type="Masonry Veneer", 
                protection_class=3, form="HO-3R", tier = 30):
        
        # Prompting for inputs with default values
        rating_area_value = int(input(f"Please enter the Rating Area [{rating_area_value}]: ") or rating_area_value)
        construction_type = input(f"Please enter the Construction Type [{construction_type}]: ") or construction_type
        protection_class = int(input(f"Please enter the Protection Class [{protection_class}]: ") or protection_class)
        form = input(f"Please enter the Form [{form}]: ") or form
        tier = int(input(f"Please enter the Form [{tier}]: ") or tier)
        

        # Running the calculation function with the provided inputs
        result = calculate_insurance_premium(rating_area_value, construction_type, 
                                             protection_class, form, tier)

        return result.values.sum()

In [ ]:
user_inputs()

In [164]:
import tkinter as tk
from tkinter import ttk, messagebox
# Function for insurance premium calculation
def calculate_insurance_premium(rating_area_value, construction_type, protection_class , form, tier):
    # Filter the row for the specified rating area in df_rating_area
    rating_area_row = territory_factor[territory_factor['Rating Area'] == rating_area_value]
    
    # Filter the row for the specified construction type and protection class in df_fire_protection
    fire_protection_class_construction_factor_row = fire_protection_class_construction_factor[(fire_protection_class_construction_factor['Construction Type'] == construction_type) & 
                                        (fire_protection_class_construction_factor['Protection Class'] == protection_class)]
    
    form_factor_row = form_factor[form_factor['Form'] == form]

    tier_factor_row = tier_factor[tier_factor['Tier'] == tier]

    # Check if both rows exist
    if not rating_area_row.empty and not fire_protection_class_construction_factor_row.empty and not form_factor_row.empty and not tier_factor_row.empty :
        # Dropping non-numeric columns to get values for multiplication
        rating_area_values = rating_area_row.drop(columns="Rating Area").iloc[0]
        fire_protection_class_construction_factor_values = fire_protection_class_construction_factor_row.drop(columns=["Construction Type", "Protection Class"]).iloc[0]
        form_factor_values = form_factor_row.drop(columns="Form").iloc[0]
        tier_factor_values =  tier_factor_row.drop(columns=["Tier", "Other Optional Coverages"]).iloc[0]

        # Calculating final values by multiplying rating area, protection factors, and transposed DataFrame values
        result = rating_area_values * fire_protection_class_construction_factor_values * form_factor_values * tier_factor_values * base_rate.iloc[0]
        
        dataframe = result.to_frame().T
        return dataframe
    

# GUI for the insurance premium calculation
def show_gui():
    def calculate_premium():
        try:
            rating_area_value = int(rating_area_entry.get())
            construction_type = construction_type_combo.get()
            protection_class = int(protection_class_entry.get())
            form = form_combo.get()
            tier = int(tier_entry.get())
            
            # Calculate the result
            result = calculate_insurance_premium(rating_area_value, construction_type, protection_class, form, tier)
            result = result.values.sum()
            if result:
                result_label.config(text=f"Calculated Premium: ${result:.2f}")
            else:
                messagebox.showerror("Error", "Invalid inputs. Please check your entries.")
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numbers for Rating Area, Protection Class, and Tier.")
    
    # Set up the window
    window = tk.Tk()
    window.title("Insurance Premium Calculator")
    window.geometry("400x300")
    
    # Input fields
    tk.Label(window, text="Rating Area").grid(row=0, column=0, padx=10, pady=10)
    rating_area_entry = tk.Entry(window)
    rating_area_entry.insert(0, "701")  # Default value
    rating_area_entry.grid(row=0, column=1)
    
    tk.Label(window, text="Construction Type").grid(row=1, column=0, padx=10, pady=10)
    construction_type_combo = ttk.Combobox(window, values=["Masonry Veneer", "Frame", "Masonry"])
    construction_type_combo.set("Masonry Veneer")  # Default value
    construction_type_combo.grid(row=1, column=1)
    
    tk.Label(window, text="Protection Class").grid(row=2, column=0, padx=10, pady=10)
    protection_class_entry = tk.Entry(window)
    protection_class_entry.insert(0, "3")  # Default value
    protection_class_entry.grid(row=2, column=1)
    
    tk.Label(window, text="Form").grid(row=3, column=0, padx=10, pady=10)
    form_combo = ttk.Combobox(window, values=["HO-3R", "HO-9R"])
    form_combo.set("HO-3R")  # Default value
    form_combo.grid(row=3, column=1)
    
    tk.Label(window, text="Tier").grid(row=4, column=0, padx=10, pady=10)
    tier_entry = tk.Entry(window)
    tier_entry.insert(0, "30")  # Default value
    tier_entry.grid(row=4, column=1)
    
    # Calculate button and result display
    calculate_button = tk.Button(window, text="Calculate Premium", command=calculate_premium)
    calculate_button.grid(row=5, column=0, columnspan=2, pady=20)
    
    
    result_label = tk.Label(window, text="Calculated Premium: $0.00")
    result_label.grid(row=6, column=0, columnspan=2, pady=10)
    
    # Run the window
    window.mainloop()

# Show the GUI
show_gui()


## Preserve this

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
# Function for insurance premium calculation

def amount_of_insurance_deductible_factor(amount_of_insurance_df, amount_of_insurance, deductible, aep):
    # Define the specific perils to filter
    perils = [
        "Other Perils", "Fire", "Other Wind and Hail", "Theft", "Water",
        "Liability", "Hurricane", "Severe Thunderstorm", "Winter Storm", "Earthquake"
    ]
    
    # Initialize the result DataFrame with perils as columns and a single row for the values
    result = pd.DataFrame(columns=perils, index=[0])
    
    # Filter the dataframe based on the given amount_of_insurance
    filtered_df = df[(df["Amount of Insurance"] == amount_of_insurance) & (df["Peril"].isin(perils))]
    
    # Iterate through each peril to retrieve the appropriate value based on the deductible and aep columns
    for peril in perils:
        # Filter the row for the current peril
        peril_row = filtered_df[filtered_df["Peril"] == peril]
        
        # Check if the peril_row is non-empty
        if not peril_row.empty:
            # First try to retrieve the value from the deductible column
            value = peril_row[deductible].values[0] if deductible in peril_row.columns else np.nan
            
            # If the deductible value is NaN, try the aep column
            if pd.isna(value) and aep in peril_row.columns:
                value = peril_row[aep].values[0]
            
            # If both deductible and aep are NaN, take the value from the "10%" column
            if pd.isna(value) and "10%" in peril_row.columns:
                value = peril_row["10%"].values[0]
            
            # Assign the final value to the result DataFrame
            result[peril].iloc[0] = value
    
    return result 


def calculate_insurance_premium(rating_area_value, construction_type, protection_class , form, tier, df, amount_of_insurance,
                                deductible, aep):
    # Filter the row for the specified rating area in df_rating_area
    rating_area_row = territory_factor[territory_factor['Rating Area'] == rating_area_value]
    
    # Filter the row for the specified construction type and protection class in df_fire_protection
    fire_protection_class_construction_factor_row = fire_protection_class_construction_factor[(fire_protection_class_construction_factor['Construction Type'] == construction_type) & 
                                        (fire_protection_class_construction_factor['Protection Class'] == protection_class)]
    
    form_factor_row = form_factor[form_factor['Form'] == form]

    tier_factor_row = tier_factor[tier_factor['Tier'] == tier]

    # Check if both rows exist
    if not rating_area_row.empty and not fire_protection_class_construction_factor_row.empty and not form_factor_row.empty and not tier_factor_row.empty :
        # Dropping non-numeric columns to get values for multiplication
        rating_area_values = rating_area_row.drop(columns="Rating Area").iloc[0]
        fire_protection_class_construction_factor_values = fire_protection_class_construction_factor_row.drop(columns=["Construction Type", "Protection Class"]).iloc[0]
        form_factor_values = form_factor_row.drop(columns="Form").iloc[0]
        tier_factor_values =  tier_factor_row.drop(columns=["Tier", "Other Optional Coverages"]).iloc[0]

        amount_of_insurance_deductible_factor_values = amount_of_insurance_deductible_factor(df, amount_of_insurance, deductible, aep)

        # Calculating final values by multiplying rating area, protection factors, and transposed DataFrame values
        result = rating_area_values * fire_protection_class_construction_factor_values * form_factor_values * tier_factor_values * amount_of_insurance_deductible_factor_values  * base_rate.iloc[0]
        
        dataframe = result.T

        return dataframe
    

# GUI for the insurance premium calculation
def show_gui():
    def calculate_premium():
        try:
            rating_area_value = int(rating_area_entry.get())
            construction_type = construction_type_combo.get()
            protection_class = int(protection_class_entry.get())
            form = form_combo.get()
            tier = int(tier_entry.get())
            df = 
            
            # Calculate the result
            result = calculate_insurance_premium(rating_area_value, construction_type, protection_class, form, tier)
            result = result.values.sum()
            if result:
                result_label.config(text=f"Calculated Premium: ${result:.2f}")
            else:
                messagebox.showerror("Error", "Invalid inputs. Please check your entries.")
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numbers for Rating Area, Protection Class, and Tier.")
    
    # Set up the window
    window = tk.Tk()
    window.title("Insurance Premium Calculator")
    window.geometry("400x300")
    
    # Input fields
    tk.Label(window, text="Rating Area").grid(row=0, column=0, padx=10, pady=10)
    rating_area_entry = tk.Entry(window)
    rating_area_entry.insert(0, "701")  # Default value
    rating_area_entry.grid(row=0, column=1)
    
    tk.Label(window, text="Construction Type").grid(row=1, column=0, padx=10, pady=10)
    construction_type_combo = ttk.Combobox(window, values=["Masonry Veneer", "Frame", "Masonry"])
    construction_type_combo.set("Masonry Veneer")  # Default value
    construction_type_combo.grid(row=1, column=1)
    
    tk.Label(window, text="Protection Class").grid(row=2, column=0, padx=10, pady=10)
    protection_class_entry = tk.Entry(window)
    protection_class_entry.insert(0, "3")  # Default value
    protection_class_entry.grid(row=2, column=1)
    
    tk.Label(window, text="Form").grid(row=3, column=0, padx=10, pady=10)
    form_combo = ttk.Combobox(window, values=["HO-3R", "HO-9R"])
    form_combo.set("HO-3R")  # Default value
    form_combo.grid(row=3, column=1)
    
    tk.Label(window, text="Tier").grid(row=4, column=0, padx=10, pady=10)
    tier_entry = tk.Entry(window)
    tier_entry.insert(0, "30")  # Default value
    tier_entry.grid(row=4, column=1)
    
    
    tk.Label(window, text="Amount of insurance").grid(row=5, column=0, padx=10, pady=10)
    amount_of_insurance_combo = ttk.Combobox(window, values=["$10,000", "$50,000", "$100,000",
                                                             "$150,000", "$200,000", "$250,000",
                                                             "$300,000", "350,000", "$400,000",
                                                             "$450,000", "$500,000", "$600,000", "750,000",
                                                             "$1,000,000", "$1,500,000", "$2,000,000",
                                                             "$3,000,000"])
    amount_of_insurance_combo.set("$200,000")  # Default value
    amount_of_insurance_combo.grid(row=5, column=1)
    
    tk.Label(window, text="Deductible").grid(row=6, column=0, padx=10, pady=10)
    deductible_combo = ttk.Combobox(window, values=["$500", "$1000", "$2000", "$5000", "$10000"])
    form_combo.set("$500")  # Default value
    form_combo.grid(row=6, column=1)
    
    tk.Label(window, text="Annual Exceedence Probability").grid(row=7, column=0, padx=10, pady=10)
    aep_combo = ttk.Combobox(window, values=["1%", "2%", "3%", "5%", "10%"])
    form_combo.set("5%")  # Default value
    form_combo.grid(row=7, column=1)

    # Calculate button and result display
    calculate_button = tk.Button(window, text="Calculate Premium", command=calculate_premium)
    calculate_button.grid(row=5, column=0, columnspan=2, pady=20)
    
    
    result_label = tk.Label(window, text="Calculated Premium: $0.00")
    result_label.grid(row=6, column=0, columnspan=2, pady=10)
    
    # Run the window
    window.mainloop()

# Show the GUI
show_gui()


In [232]:
insurance_to_value = 0.95

In [236]:
insurance_to_value_table = pd.read_csv("D:\\homeowner_insurance_USAA\\output_tables\\Owner\\30_insurance_to_value.csv")

In [237]:
insurance_to_value_row = insurance_to_value_table[insurance_to_value_table['ITV Percent'] == insurance_to_value]
insurance_to_value_values = insurance_to_value_row.drop(columns="ITV Percent").iloc[0]

In [238]:
insurance_to_value_values 

Other Perils           0.95
Fire                   0.95
Other Wind and Hail    0.95
Theft                  0.95
Water                  0.95
Liability              0.95
Hurricane              0.95
Severe Thunderstorm    0.95
Winter Storm           0.95
Earthquake             0.95
Name: 20, dtype: float64